In [1]:
### read GEDI 
import os
import h5py
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pyarrow
import glob
import json ## file format

In [3]:
def gedi2parquet(gediL2A):

    df = pd.DataFrame()
    beamNames = [g for g in gediL2A.keys() if g.startswith('BEAM')]
    if not beamNames: 
        return None
    for b in beamNames:
        #print(b)
        # Open the SDS
        result = pd.DataFrame()
        if 'beam' not in gediL2A[f'{b}'].keys(): continue
        result['beam']= gediL2A[f'{b}/beam'][:]
        result['degrade_flag']=gediL2A[f'{b}/degrade_flag'][:]
        result['delta_time']=gediL2A[f'{b}/delta_time'][:]
        result['lat_lowestmode'] = gediL2A[f'{b}/lat_lowestmode'][:]
        result['lon_lowestmode'] = gediL2A[f'{b}/lon_lowestmode'][:]
        result['quality_flag'] = gediL2A[f'{b}/quality_flag'][:]
    ## rh 2d metrics
        rh_2d = gediL2A[f'{b}/rh'][:][:]
        col_list = []
        for j,_ in enumerate(rh_2d[0]): ##iterate each row
            col_list.append('rh_' + str(j))
            #result['rh_' + str(j)] = rh_2d[:,j]
        rh_df = pd.DataFrame(rh_2d, columns=col_list) 
        
        #rh_df = rh_df.div(100) # convert cm to m in unit.
    #print(rh_df)
        result = pd.concat([result, rh_df], axis=1)
        result['sensitivity'] = gediL2A[f'{b}/sensitivity'][:]
        result['shot_number'] = gediL2A[f'{b}/shot_number'][:]
        result['solar_elevation'] = gediL2A[f'{b}/solar_elevation'][:]
        result['landsat_treecover'] = gediL2A[f'{b}/land_cover_data/landsat_treecover'][:]
        result.insert(0, 'name', b)
        if df.empty:
            df = result
        else: 
            df = pd.concat([df, result])
        # Create a geometry column using Latitude and Longitude columns
        geometry = [Point(xy) for xy in zip(df['lon_lowestmode'], df['lat_lowestmode'])]

        # Create a GeoDataFrame from the DataFrame and geometry column
        gdf = gpd.GeoDataFrame(df, geometry=geometry)

        # Set the CRS of the GeoDataFrame to WGS84 (EPSG:4326)
        gdf.crs = 'EPSG:4326'
        return gdf

In [4]:
folder_path = '/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/gedi'  # Replace with the actual folder path
# Use the glob function to find files with ".h5" extension in the given folder path
h5_files = glob.glob(folder_path + '/*.h5')
result = pd.DataFrame()
# Print the list of h5 files
for file in h5_files:
    #print(file)
    gediL2A = h5py.File(file, 'r')  # Read file using h5py
    gdf = gedi2parquet(gediL2A)
    # Write the GeoDataFrame to a Parquet file
    # Get the base name without extension
    base_name_without_extension = os.path.splitext(os.path.basename(file))[0]
    file_path = '../out/' + base_name_without_extension  + '.parquet'
    #gdf.to_parquet(file_path)
    result = pd.concat([result, gdf])
#result.to_parquet('../out/gedi_merge.parquet')

In [5]:
len(result)

338343

In [6]:
from pyproj import Transformer  ## proj projection, and transformation.
df_valid = result

### apply filter
def get_tile_id(longitude, latitude, tilesize=72):
    ease2_origin = -17367530.445161499083042, 7314540.830638599582016
    ease2_nbins = int(34704 / tilesize), int(14616 / tilesize)
    ease2_binsize = 1000.895023349556141*tilesize, 1000.895023349562052*tilesize 
    transformer = Transformer.from_crs('epsg:4326', 'epsg:6933', always_xy=True)
    x,y = transformer.transform(longitude, latitude)
    xidx = np.uint16( (x - ease2_origin[0]) / ease2_binsize[0]) + 1
    yidx = np.uint16( (ease2_origin[1] - y) / ease2_binsize[1]) + 1
    tile_id = np.array([f'X{x:03d}Y{y:03d}' for x,y in zip(xidx,yidx)])
    return tile_id

fn = '/gpfs/data1/vclgp/armstonj/l4b_granule_filtering/issgedi_l4b_excluded_granules_r002_20230315a_plusManual.json'
with open(fn, 'r') as f:
    excluded_granules = json.load(f)

tile_id = get_tile_id(df_valid['lon_lowestmode'].to_numpy(), df_valid['lat_lowestmode'].to_numpy())
orbit = (df_valid.index // 10000000000000)
granule = (df_valid.index % 100000000000) // 100000000
granule_id = orbit * 10 + granule

granule_mask = np.array([(True if g in excluded_granules[t] else False) if t in excluded_granules else False
                         for g,t in zip(granule_id,tile_id)])

df_valid = df_valid.assign(granule_mask=granule_mask)

In [7]:
df_quality = df_valid.query('(`sensitivity` > 0.7) and \
                            (`sensitivity` < 1) and \
                            (`granule_mask` != True) and \
                            (`quality_flag` == 1) and \
                            (`degrade_flag` == 0) ')
len(df_quality)
df_quality

,name,beam,degrade_flag,delta_time,lat_lowestmode,lon_lowestmode,quality_flag,rh_0,rh_1,rh_2,...,rh_97,rh_98,rh_99,rh_100,sensitivity,shot_number,solar_elevation,landsat_treecover,geometry,granule_mask
0,BEAM0000,0,0,1.318078e+08,26.129798,-81.247669,1,-4.94,-4.38,-3.93,...,12.77,13.110000,13.59,14.190000,0.953535,182940000300251676,19.517761,47.0,POINT (-81.24767 26.12980),False
1,BEAM0000,0,0,1.318078e+08,26.129410,-81.247294,1,-3.29,-3.07,-2.84,...,11.80,12.210000,12.66,13.180000,0.914217,182940000300251677,19.518225,45.0,POINT (-81.24729 26.12941),False
4,BEAM0000,0,0,1.318078e+08,26.128245,-81.246167,1,-4.04,-3.59,-3.22,...,3.07,3.400000,3.78,4.270000,0.940894,182940000300251680,19.519613,47.0,POINT (-81.24617 26.12825),False
6,BEAM0000,0,0,1.318078e+08,26.127468,-81.245416,1,-3.85,-3.40,-2.99,...,16.59,17.000000,17.42,17.900000,0.945859,182940000300251682,19.520542,47.0,POINT (-81.24542 26.12747),False
9,BEAM0000,0,0,1.318078e+08,26.126303,-81.244290,1,-3.70,-3.29,-3.03,...,6.93,7.260000,7.64,8.050000,0.958460,182940000300251685,19.521933,45.0,POINT (-81.24429 26.12630),False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,BEAM0000,0,0,5.961181e+07,26.128486,-81.400260,1,-4.00,-3.36,-2.84,...,13.36,13.890000,14.45,15.080000,0.973441,53410000200066829,-3.927317,78.0,POINT (-81.40026 26.12849),False
1195,BEAM0000,0,0,5.961181e+07,26.128874,-81.399884,1,-5.27,-4.41,-3.66,...,16.84,17.260000,17.74,18.420000,0.966876,53410000200066830,-3.927796,74.0,POINT (-81.39988 26.12887),False
1196,BEAM0000,0,0,5.961181e+07,26.129262,-81.399509,1,-2.73,-2.43,-2.17,...,19.43,19.799999,20.25,20.809999,0.910555,53410000200066831,-3.928276,86.0,POINT (-81.39951 26.12926),False
1197,BEAM0000,0,0,5.961181e+07,26.129651,-81.399133,1,-3.03,-2.73,-2.47,...,15.42,15.800000,16.25,16.920000,0.908440,53410000200066832,-3.928758,77.0,POINT (-81.39913 26.12965),False


In [9]:
# convert delta time to date
from datetime import datetime, timedelta

# get power beam flag
# List of beams that should get a new column value of 1
target_beams = ['BEAM0101', 'BEAM0110', 'BEAM1000', 'BEAM1011']
# Using apply() with a lambda function
df_quality['power_flag'] = df_quality['name'].apply(lambda x: 1 if x in target_beams else 0)




# Convert the 'seconds_since_2018' column to pandas datetime objects
df_quality['date'] = pd.to_datetime(df_quality['delta_time'], 
                                    unit='s', origin='2018-01-01') \
                     .dt.strftime('%Y-%m-%d')
# convert solar elevation to night flag

df_quality['night_flag'] = df_quality['solar_elevation'].apply(lambda x: 1 if x < 0 else 0)
df_quality

df_quality.to_parquet('../out/gedi_merge.parquet')
df_quality.to_csv('../out/gedi_merge.csv', index=False)

/home/xiongl/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [8]:
df_quality.total_bounds ## get bounds for gmt gridding.

array([-81.84877916,  24.93065147, -80.30657902,  26.13000322])

In [19]:
# filter by mangrove mask ## seems not working here.

# import rasterio
# from pyproj import Transformer

# gdf = df_quality.copy()
# gdf = gdf.reset_index(drop=True)
# import rasterio
# import numpy as np
# import warnings
# warnings.filterwarnings("ignore")
# # Load the TIFF image using rasterio
# file = "southFlorida_MangroveResilience_Irma_wgs84.tif"
# # Read the GeoTIFF file
# ndviRaster = rasterio.open(file)
#     # Extract raster values for all points in the GeoDataFrame
# xmin, ymin, xmax, ymax = ndviRaster.bounds  
#     #extract point value from raster
# gdf['mangrove'] = -1
# for index, row in gdf.iterrows():
#     x = row['geometry'].xy[0][0]
#     y = row['geometry'].xy[1][0]
#     #print(x, y)
#     row, col = ndviRaster.index(x,y)
#     if x>xmin and x<xmax and y>ymin and y<ymax:
#             value = ndviRaster.read(1)[row,col]
#             gdf.loc[index, 'mangrove'] = value
#     #print("Point correspond to row, col: %d, %d"%(row,col))
#     #print("Raster value on point %.2f \n"%ndviRaster.read(1)[row,col])
# # Print the updated GeoDataFrame with the raster values
# print(gdf)

BoundingBox(left=-82.218194444, bottom=24.481527778, right=-80.367361111, top=26.078472222)

In [4]:
####### Read IS2 data
#!/usr/bin/env python
# coding: utf-8

#import icepyx as ipx

import numpy as np

import xarray as xr

import pandas as pd

import h5py

import os,json

from pprint import pprint

import dask.dataframe as dd
import csv
import glob
# put the full filepath to a data file here. You can get this in JupyterHub by navigating to the file,

# right clicking, and selecting copy path. Then you can paste the path in the quotes below.
#import ATL08_plot


def read_atl08(fname):


###  Extract TIme Beam lat lon h_canopy h_te_mean 
        with h5py.File(fname, mode='r') as fi:
                #print("processing file: " + str(fi))
                # table initial
                df = pd.DataFrame()
                #print("Keys: %s" % fi.keys())       
                time = fi['/ancillary_data/data_start_utc'][0][0:10].decode('UTF-8')
                sc_orient  =  fi['orbit_info/sc_orient'][0]
                # Each beam is a group
                group = ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
                # Loop trough beams
                for k,g in enumerate(group):
                    # 1) Read in data for a single beam #

                    #-----------------------------------#

                    # Load variables into memory (more can be added!)
                    # if beam not exist 
                   
                    if g[1:] not in list(fi.keys())  : continue
                            
                                  
                    # if land segment not exist 
                    if 'land_segments' not in list(fi[g].keys()):  continue

                    result = pd.DataFrame()

                                  
                    result['lat'] = fi[g+'/land_segments/latitude'][:]

                    result['lon'] = fi[g+'/land_segments/longitude'][:]

                    
                    ## lat 2d metrics
                    lat_2d = fi[g+'/land_segments/latitude_20m'][:][:]
                    col_list = []
                    for j,_ in enumerate(lat_2d[0]): ##iterate each row
                                col_list.append('lat_20m_' + str(j).zfill(3))
                                #result['rh_' + str(j)] = rh_2d[:,j]
                    lat_df = pd.DataFrame(lat_2d, columns=col_list)
                    result = pd.concat([result, lat_df], axis=1)                    
                    ## lon 2d metrics
                    lon_2d = fi[g+'/land_segments/longitude_20m'][:][:]
                    col_list = []
                    for j,_ in enumerate(lon_2d[0]): ##iterate each row
                                col_list.append('lon_20m_' + str(j).zfill(3))
                                #result['rh_' + str(j)] = rh_2d[:,j]
                    lon_df = pd.DataFrame(lon_2d, columns=col_list)
                    result = pd.concat([result, lon_df], axis=1)                        
                    
                    
                    
                    

                    result['h_canopy'] = fi[g+'/land_segments/canopy/h_canopy'][:]
                    ## rh 2d metrics
                    h_2d = fi[g+'/land_segments/canopy/h_canopy_20m'][:][:]
                    col_list = []
                    for j,_ in enumerate(h_2d[0]): ##iterate each row
                                col_list.append('h_canopy_20m_' + str(j).zfill(3))
                                #result['rh_' + str(j)] = rh_2d[:,j]
                    h_df = pd.DataFrame(h_2d, columns=col_list)
                    result = pd.concat([result, h_df], axis=1)
                    
                    ### add uncertainty
                    result['h_canopy_uncertainty']  = fi[g+'/land_segments/canopy/h_canopy_uncertainty'][:]
                
                    result['h_te_mean'] =  fi[g+'/land_segments/terrain/h_te_mean'][:]

                    result['h_te_best_fit ']  =  fi[g+'/land_segments/terrain/h_te_best_fit'][:]

                    result['h_te_uncertainty']  =  fi[g+'/land_segments/terrain/h_te_uncertainty'][:]
                    result['night_flag']  =  fi[g+'/land_segments/night_flag'][:]
                    result['n_ca_photons']  =  fi[g+'/land_segments/canopy/n_ca_photons'][:]
                    result.insert(0, 'beam', g[1:])
                   # result['beam'] = g[1:]
                   # extract time
                    result.insert(1, 'time', time)
                    result.insert(2, 'sc_orient', sc_orient)
                    
                    #print(result)
                    frames = [df, result]
                    ##update df
                    df = pd.concat(frames)
                    #rows.append(result)
                    #print(df)
                    #print(pd.concat([rows, result]))
                   # rows = pd.concat(rows)
                    #print(pd.concat([rows, result]))
                    
                #print(fname[-49:-3])
                #df.to_csv('CSV/' + fname[-49:-3] +'.csv', index=False)
                # Create a geometry column using Latitude and Longitude columns
        geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

        # Create a GeoDataFrame from the DataFrame and geometry column
        gdf = gpd.GeoDataFrame(df, geometry=geometry)

        # Set the CRS of the GeoDataFrame to WGS84 (EPSG:4326)
        gdf.crs = 'EPSG:4326'
        
        
        return gdf




In [6]:
f='/gpfs/data1/vclgp/xiongl/ProjectFlorida/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20190613212415_11700307_006_02.h5'
df = read_atl08(f)
df.head(10)

,beam,time,sc_orient,lat,lon,lat_20m_000,lat_20m_001,lat_20m_002,lat_20m_003,lat_20m_004,...,h_canopy_20m_002,h_canopy_20m_003,h_canopy_20m_004,h_canopy_uncertainty,h_te_mean,h_te_best_fit,h_te_uncertainty,night_flag,n_ca_photons,geometry
0,gt1l,2019-06-13,0,25.143667,-80.297775,25.144028,25.143848,25.143667,25.143488,25.143309,...,8.875153e+00,7.696295e+00,8.236185e+00,2.778358e-01,-28.829700,-31.640953,0.468531,0,27,POINT (-80.29778 25.14367)
1,gt1l,2019-06-13,0,25.142767,-80.297874,25.143126,25.142946,25.142767,25.142588,25.142408,...,9.274300e+00,8.255047e+00,9.294416e+00,2.952636e-01,-34.183472,-34.121574,0.743146,0,30,POINT (-80.29787 25.14277)
2,gt1l,2019-06-13,0,25.141867,-80.297974,25.142225,25.142046,25.141867,25.141687,25.141506,...,8.203262e+00,8.093664e+00,8.493584e+00,2.265133e-01,-33.732525,-33.852837,0.733374,0,14,POINT (-80.29797 25.14187)
3,gt1l,2019-06-13,0,25.140965,-80.298065,25.141325,25.141144,25.140965,25.140785,25.140606,...,7.758732e-01,3.196911e+00,7.624014e+00,3.185387e-01,-28.960506,-26.266590,0.485496,0,52,POINT (-80.29807 25.14096)
4,gt1l,2019-06-13,0,25.140064,-80.298164,25.140423,25.140244,25.140064,25.139885,25.139706,...,6.439251e+00,6.167194e+00,6.206900e+00,2.295623e-01,-32.379112,-32.462376,0.437568,0,20,POINT (-80.29816 25.14006)
5,gt1l,2019-06-13,0,25.138262,-80.298363,25.138622,25.138441,25.138262,25.138083,25.137903,...,9.684093e+00,8.493828e+00,8.404978e+00,3.315407e-01,-30.033449,-33.237076,0.522742,0,37,POINT (-80.29836 25.13826)
6,gt1l,2019-06-13,0,25.137362,-80.298462,25.137720,25.137541,25.137362,25.137182,25.137003,...,8.926134e+00,4.474703e+00,3.402823e+38,3.275603e-01,-27.873922,-35.579594,0.597517,0,19,POINT (-80.29846 25.13736)
7,gt1l,2019-06-13,0,25.136461,-80.298561,25.136820,25.136641,25.136461,25.136282,25.136101,...,3.402823e+38,3.402823e+38,3.402823e+38,3.402823e+38,-26.372154,-26.371845,0.187041,0,0,POINT (-80.29856 25.13646)
8,gt1l,2019-06-13,0,25.135559,-80.298653,25.135920,25.135740,25.135559,25.135380,25.135201,...,3.402823e+38,3.402823e+38,3.402823e+38,3.402823e+38,-26.399012,-26.390408,0.225639,0,0,POINT (-80.29865 25.13556)
9,gt1l,2019-06-13,0,25.134659,-80.298752,25.135017,25.134838,25.134659,25.134480,25.134300,...,3.402823e+38,3.402823e+38,3.402823e+38,3.402823e+38,-27.382793,-25.991348,0.309582,0,0,POINT (-80.29875 25.13466)


In [11]:
folder_path = '/gpfs/data1/vclgp/xiongl/ProjectFlorida/FloridaMangrove/EARTHDATASEARCH/atl08'  # Replace with the actual folder path

# Use the glob function to find files with ".h5" extension in the given folder path
h5_files = glob.glob(folder_path + '/*.h5')
    
total = pd.DataFrame()
for fname in h5_files:
    print(fname)
    gdf = read_atl08(fname)
    # plot data
    base_name_without_extension = os.path.splitext(os.path.basename(fname))[0]
    file_path = '../out/' + base_name_without_extension  + '.parquet'
    #gdf.to_parquet(file_path)
    #ATL08_plot.ATL08plot(path+'//'+fname)
    #print(result)
    ##update df
    total = pd.concat([total, gdf])

#total
### plot
#print(total['lon'])
#ATL08_plot.ATL08plot(total, 'IS_all') 

#total.to_csv('CSV/' + 'total_update' +'.csv', index=False)

/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20200804131809_06140801_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20220914124542_12921607_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20210729200555_05531201_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20220102124121_01721401_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20190218030016_07890207_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20230328150843_01111901_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20190613212415_11700307_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20181214060440_11700107_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20190319013620_12

/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20230409024932_02861907_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20220830011322_10561601_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20211225125800_00501401_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20230214052912_08501807_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20190916165546_12310407_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20190807063856_06140401_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20210401014959_01111101_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20190621210735_12920307_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20200913233455_12

/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20221001234100_01721701_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20220326083753_00501501_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20210416132223_03471107_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20220703040110_01721601_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20200718022252_03470807_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20221013112140_03471707_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20201206072542_11170901_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20200902115411_10560801_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20181103194803_05

/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20201213191446_12310907_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20200418064304_03470707_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20220301095328_10561401_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20190519223954_07890307_006_02.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20200402191037_01110701_006_03.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20200114111135_02860607_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20221210083340_12311707_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20220305094508_11171401_006_01.h5
/gpfs/data1/vclgp/xiongl/FloridaMangrove/EARTHDATASEARCH/atl08/processed_ATL08_20221107100602_07

In [24]:
len(total)
#total.head(1)

1024789

In [12]:
### quality filter here 
'''
### I-2 filter:
atl08 = atl08[(atl08$h_te_uncertainty < 3.402823e+23 | atl08$h_te_uncertainty > -999),]
atl08 = atl08[(atl08$h_te_best_fit < 3.402823e+23 | atl08$h_te_best_fit > -999),]
atl08 = atl08[(atl08$h_te_median < 3.402823e+23 | atl08$h_te_median > -999),]
atl08 = atl08[(atl08$h_canopy > 0 & atl08$h_canopy < 75),]
atl08 = atl08[(atl08$n_ca_photons < 140),]
+ strong beams + night
'''
total_quality = total.query('(`h_canopy` > 0) and \
                            (`h_canopy` < 50) and \
                            (`h_canopy_uncertainty` < 50) and \
                            (`h_te_mean` < 100) and \
                            (`h_te_mean` > -999) and \
                            (`n_ca_photons` > 1) and \
                            (`n_ca_photons` < 140)\
                            ')

In [13]:
len(total_quality)
#total_quality.head(1)

259380

In [15]:
# get 20 m product 
df = total_quality
geo_cols = {'h_canopy_20m_000': 'h_canopy_20m',
            'h_canopy_20m_001': 'h_canopy_20m',
            'h_canopy_20m_002': 'h_canopy_20m',
            'h_canopy_20m_003': 'h_canopy_20m',
            'h_canopy_20m_004': 'h_canopy_20m',
            'lat_20m_000': 'lat_20m',
            'lat_20m_001': 'lat_20m',
            'lat_20m_002': 'lat_20m',
            'lat_20m_003': 'lat_20m',
            'lat_20m_004': 'lat_20m',
            'lon_20m_000': 'lon_20m',
            'lon_20m_001': 'lon_20m',
            'lon_20m_002': 'lon_20m',
            'lon_20m_003': 'lon_20m',
            'lon_20m_004': 'lon_20m',
            'h_te_best_fit_20m_000': 'h_te_best_fit_20m',
            'h_te_best_fit_20m_001': 'h_te_best_fit_20m',
            'h_te_best_fit_20m_002': 'h_te_best_fit_20m',
            'h_te_best_fit_20m_003': 'h_te_best_fit_20m',
            'h_te_best_fit_20m_004': 'h_te_best_fit_20m',
           }
#df = df.reset_index(drop=True) 
## filter first
df_000 = df.drop(columns=df.columns[df.columns.str.contains('001|002|003|004')])
df_000 = df_000.rename(columns=geo_cols)

##### data 001
df_001 = df.drop(columns=df.columns[df.columns.str.contains('000|002|003|004')])
df_001 = df_001.rename(columns=geo_cols) 

##### data 002
df_002 = df.drop(columns=df.columns[df.columns.str.contains('000|001|003|004')])
df_002 = df_002.rename(columns=geo_cols) 

##### data 003
df_003 = df.drop(columns=df.columns[df.columns.str.contains('000|001|002|004')])
df_003 = df_003.rename(columns=geo_cols) 

##### data 004
df_004 = df.drop(columns=df.columns[df.columns.str.contains('000|001|002|003')])
df_004 = df_004.rename(columns=geo_cols) 
#print(df_000.dtypes)
# Get the current column names


#print(df_004.columns)
###########
df_new =  pd.concat([df_000, df_001, df_002,df_003,df_004 ])
df_new = df_new.reset_index(drop=True)  
#### condunct geo refrence 
# Create a GeoDataFrame from the pandas DataFrame
geometry = [Point(xy) for xy in zip(df_new['lon_20m'], df_new['lat_20m'])]
gdf = gpd.GeoDataFrame(df_new , geometry=geometry,crs='EPSG:4326')

In [16]:
len(gdf)

1296900

In [17]:

total_quality = gdf.query('(`h_canopy_20m` > 0) and \
                            (`h_canopy_20m` < 50) and \
                            (`lat_20m` < 181) and \
                            (`lon_20m` < 181)\
                            ')

In [18]:
len(total_quality)
#total_quality.dtypes
#total_quality.head(1)

858095

In [31]:
### get strong or weak beam flag
'''
forward  orientation: strong:  gt1r, gt2r, gt3r
backward orientation: strong:  gt1l, gt2l, gt3l
Flag Values: ['0', '1', '2']
Flag Meanings: ['backward', 'forward', 'transition']
'''
# Function to apply the conditions and create a new column
def create_new_column(row):
    if (row['sc_orient'] == 1 and row['beam'] in ['gt1r', 'gt2r', 'gt3r']) \
        or (row['sc_orient'] == 0 and row['beam'] in ['gt1l', 'gt2l', 'gt3l']):
        return 1
    else:
        return 0

# Apply the function to create the new column
total_quality['strong_flag'] = total_quality.apply(create_new_column, axis=1)
print(total_quality)


         beam        time  sc_orient        lat        lon    lat_20m  \
4        gt1l  2020-08-04          0  25.786442 -81.415871  25.786081   
5        gt1l  2020-08-04          0  25.805359 -81.417923  25.804998   
6        gt1l  2020-08-04          0  25.811665 -81.418602  25.811304   
7        gt1l  2020-08-04          0  25.822474 -81.419777  25.822113   
8        gt1l  2020-08-04          0  25.823374 -81.419876  25.823015   
...       ...         ...        ...        ...        ...        ...   
1296884  gt3r  2019-12-16          1  24.982111 -81.028427  24.981750   
1296888  gt3r  2019-12-16          1  24.972198 -81.029495  24.971840   
1296889  gt3r  2019-12-16          1  24.971298 -81.029587  24.970940   
1296891  gt3r  2019-12-16          1  24.965891 -81.030167  24.965532   
1296892  gt3r  2019-12-16          1  24.964991 -81.030266  24.964632   

           lon_20m   h_canopy  h_canopy_20m  h_canopy_uncertainty  h_te_mean  \
4       -81.415833  12.203262     12.340755

In [32]:
total_quality.to_csv('../out/is2_20m_merge.csv', index=False)

In [33]:
total_quality.to_parquet('../out/is2_20m_merge.parquet')

In [19]:
total_quality

,beam,time,sc_orient,lat,lon,lat_20m,lon_20m,h_canopy,h_canopy_20m,h_canopy_uncertainty,h_te_mean,h_te_best_fit,h_te_uncertainty,night_flag,n_ca_photons,geometry
4,gt1l,2020-08-04,0,25.786442,-81.415871,25.786081,-81.415833,12.203262,12.340755,0.036028,-25.006617,-24.649805,0.033126,0,90,POINT (-81.41583 25.78608)
5,gt1l,2020-08-04,0,25.805359,-81.417923,25.804998,-81.417877,8.911011,8.210073,0.047872,-24.932909,-24.747749,0.042262,0,74,POINT (-81.41788 25.80500)
6,gt1l,2020-08-04,0,25.811665,-81.418602,25.811304,-81.418564,9.814927,10.097447,0.070963,-24.828594,-24.686773,0.066390,0,63,POINT (-81.41856 25.81130)
7,gt1l,2020-08-04,0,25.822474,-81.419777,25.822113,-81.419739,11.318779,10.998100,0.034378,-25.094330,-25.151245,0.030056,0,70,POINT (-81.41974 25.82211)
8,gt1l,2020-08-04,0,25.823374,-81.419876,25.823015,-81.419838,14.459540,14.225080,0.027833,-24.699263,-24.678764,0.025864,0,60,POINT (-81.41984 25.82302)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296884,gt3r,2019-12-16,1,24.982111,-81.028427,24.981750,-81.028465,2.372511,2.372511,0.108454,-24.457829,-24.447618,0.080989,0,27,POINT (-81.02847 24.98175)
1296888,gt3r,2019-12-16,1,24.972198,-81.029495,24.971840,-81.029533,5.744884,5.524225,0.122027,-25.360291,-29.938414,0.454360,0,50,POINT (-81.02953 24.97184)
1296889,gt3r,2019-12-16,1,24.971298,-81.029587,24.970940,-81.029633,5.440588,5.440588,0.123758,-29.227657,-28.052124,2.249459,0,77,POINT (-81.02963 24.97094)
1296891,gt3r,2019-12-16,1,24.965891,-81.030167,24.965532,-81.030205,5.545935,5.545935,0.119008,-24.593782,-24.351082,0.128175,0,28,POINT (-81.03020 24.96553)


In [7]:
bool(None)

False